# Population Weight

## 0. Data Load

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/기상청/데이터')

In [4]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm.notebook import tqdm
tqdm.pandas()

import random
from functools import reduce

import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
%matplotlib inline

rc('font', family='MalgunGothic')
plt.rcParams['axes.unicode_minus'] = False

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
population = pd.read_csv('시도별 주민등록 인구현황.csv', encoding='CP949')

In [6]:
population.head(2)

,시·도별(1),2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7,2018.8,2018.9,2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7,2019.8,2019.9
0,시·도별(1),총인구 (명),남 (명),여 (명),한국인_세대 (세대),한국인_계 (명),한국인_남 (명),한국인_여 (명),외국인_계 (명),외국인_남 (명),외국인_여 (명),총인구 (명),남 (명),여 (명),한국인_세대 (세대),한국인_계 (명),한국인_남 (명),한국인_여 (명),외국인_계 (명),외국인_남 (명),외국인_여 (명)
1,전국,53079513,26585715,26493798,22042947,51826059,25866129,25959930,1253454,719586,533868,53128391,26595111,26533280,22481466,51849861,25864816,25985045,1278530,730295,548235


## 1. Data Preprocessing

In [7]:
population.columns

Index(['시·도별(1)', '2018', '2018.1', '2018.2', '2018.3', '2018.4', '2018.5',
       '2018.6', '2018.7', '2018.8', '2018.9', '2019', '2019.1', '2019.2',
       '2019.3', '2019.4', '2019.5', '2019.6', '2019.7', '2019.8', '2019.9'],
      dtype='object')

In [7]:
df_pop = population.loc[:,['시·도별(1)','2018','2019']]
df_population = df_pop.drop(0,axis=0).reset_index(drop=True)

In [8]:
df_population['2018'] = df_population['2018'].astype('float')
df_population['2019'] = df_population['2019'].astype('float')

In [58]:
df_population.head(2)

,시·도별(1),2018,2019
0,전국,53079513.0,53128391.0
1,서울,10049607.0,10010983.0


In [59]:
print(df_population[['시·도별(1)']])

   시·도별(1)
0       전국
1       서울
2       부산
3       대구
4       인천
5       광주
6       대전
7       울산
8       세종
9       경기
10      강원
11      충북
12      충남
13      전북
14      전남
15      경북
16      경남
17      제주


In [9]:
df_weight = pd.DataFrame(columns=['시도','가중치'])

In [29]:
df_population[df_population['시·도별(1)'].isin(['서울','경기','인천'])].loc[:,['2018','2019']]

,2018,2019
1,10049607.0,10010983.0
4,3022511.0,3029285.0
9,13485679.0,13653984.0


In [31]:
weight = []
weight.append(df_population[df_population['시·도별(1)'].isin(['서울','경기','인천'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['강원'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['경북','대구'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['경남','부산','울산'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['전남','광주'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['전북'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['충남','대전','세종'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['충북'])].loc[:,['2018','2019']].sum().mean())
weight.append(df_population[df_population['시·도별(1)'].isin(['제주'])].loc[:,['2018','2019']].sum().mean())

In [32]:
sido = ['수도권','강원','경북','경남','전남','전북','충남','충북','제주']

In [33]:
df_weight['시도'] = sido
df_weight['가중치'] = weight

In [34]:
df_weight['최종_가중치'] = df_weight['가중치']/df_weight['가중치'].sum()

In [35]:
df_weight.sort_values(by='최종_가중치',ascending=False)

,시도,가중치,최종_가중치,final_가중치
0,수도권,26626024.5,0.501394,0.39
3,경남,8095822.0,0.152452,0.12
2,경북,5207889.5,0.098070,0.11
6,충남,4028170.0,0.075854,0.06
4,전남,3390919.5,0.063854,0.07
5,전북,1860085.0,0.035027,0.08
7,충북,1639755.0,0.030878,0.07
1,강원,1560942.0,0.029394,0.07
8,제주,694344.5,0.013075,0.03


In [38]:
df_weight['final_가중치'] = round(df_weight['최종_가중치'],2)

In [39]:
df_final = df_weight[['시도','final_가중치']]

In [40]:
df_final.to_csv('/content/drive/MyDrive/기상청/데이터/population_wieght.csv', encoding='CP949', index=False)

In [41]:
pd.read_csv('/content/drive/MyDrive/기상청/데이터/population_wieght.csv', encoding='CP949')

,시도,final_가중치
0,수도권,0.50
1,강원,0.03
2,경북,0.10
3,경남,0.15
4,전남,0.06
5,전북,0.04
6,충남,0.08
7,충북,0.03
8,제주,0.01
